In [ ]:
import matplotlib       
import matplotlib.pyplot as plt
import numpy as np      
import pandas as pd     
import rushd as rd      
import scipy as sp      
import seaborn as sns   

# Set seaborn style
sns.set_style('ticks')
sns.set_context('talk', rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

There are two examples in this notebook.

1. Measuring relative expression of transcripts using RT-qPCR, example data from ComMAND
2. Quantifying DNA library prep for a genomics assay (RCMC), example data from TANGLES

# Example 1: Measuring relative expression across conditions for RT-qPCR

## Load data

In [ ]:
base_path = rd.datadir/'data'/'qPCR'/'emma'/'command'
output_path = rd.rootdir/'output'/'qPCR-example'
cache_path = output_path/'data.gzip'

plates = pd.DataFrame({
    'data_path': [base_path/'2024.11.13_command'/'2024.11.13_ELP_command_qPCR_Cp.txt', base_path/'2024.11.22_command'/'2024.11.22_ELP_command_qPCR_Cp.txt', base_path/'2024.12.03_command'/'2024.12.03_ELP_command_qPCR_Cp.txt'],
    'yaml_path': [base_path/'2024.11.13_command'/'wells.yaml', base_path/'2024.11.22_command'/'wells.yaml', base_path/'2024.12.03_command'/'wells.yaml'],
    'exp': ['exp123', 'exp123.2', 'exp123.4']
})

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
data = pd.DataFrame()
if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = rd.qpcr.load_plates_with_metadata(plates, is_default=True)
    data.to_parquet(rd.outfile(cache_path))
data.dropna(subset=['primers'], inplace=True)
display(data)

## Use controls to exclude low-quality measurements, combine technical reps

Look at controls first.

We have negative controls without any cDNA (no template for reaction) and without reverse transcriptase (for the RT reaction, so also should have no template). These set the level of "background" cycle counts, so we should treat conditions with Cp values greater than or equal to this threshold as not expressing.

As another control, we have uninfected cells ("UI"), which should have detectable levels of endogenous transcripts (GAPDH, FMRP (real name: Fmr1), and FXN) but undetectable transgenes (mRuby2, EGFP). 

In [ ]:
plot_df = data[data['construct'].isin(['UI','no-RT', 'no-cDNA'])]
g = sns.catplot(data=plot_df, x='primers', y='Cp', col='biorep', hue='construct', dodge=True)

for ax in g.axes.flatten():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

Now look at conditions of interest.

Want to look at all technical replicates, in case there are outliers that we should exclude (i.e., wells with no amplification, Cp = 35).

In [ ]:
plot_df = data[~data['construct'].isin(['UI','no-RT', 'no-cDNA'])]
g = sns.catplot(data=plot_df, x='primers', y='Cp', row='biorep', col='construct', hue='dox',
                dodge=True, margin_titles=True)
for ax in g.axes[-1,:].flatten():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

Indeed, there are a few wells with no amplification. Let's remove all these.

In [ ]:
data_filtered = data[(data['Cp']<35)].copy()

Now, combine technical replicates. Usually, we take the median.

In [ ]:
stats = data_filtered.groupby(['exp','biorep','construct','dox','primers'])[['Cp']].median().reset_index().dropna(subset='Cp')
display(stats)

In [ ]:
plot_df = stats[~stats['construct'].isin(['no-RT', 'no-cDNA'])]
g = sns.catplot(data=plot_df, x='primers', y='Cp', col='construct', hue='dox',
                dodge=True, margin_titles=True)
for ax in g.axes.flatten():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

For each set of primers, find conditions (construct + dox) where Cp values are above the no cDNA and no RT controls for a given biorep. This represents background amplification, so exclude these conditions.

Not all reps and primer sets included these negative controls. In this case, we won't exclude anything. (We achieve this easily by setting the negative control Cps to 35, assuming no amplification and thus including all other Cp values).

In [ ]:
index_on = ['exp','biorep','primers','construct']
no_cdna_control = stats.set_index(index_on)[['Cp']].xs('no-cDNA', level='construct').rename(columns={'Cp': 'Cp_no-cDNA'})
no_rt_control = stats.set_index(index_on)[['Cp']].xs('no-RT', level='construct').rename(columns={'Cp': 'Cp_no-RT'})
stats = stats.merge(no_cdna_control, on=index_on[:-1], how='left').merge(no_rt_control, on=index_on[:-1], how='left')

# not all reps + primers had these negative controls, so fill NaNs with Cp=35
stats.fillna(35, inplace=True)

# exclude conditions with high Cps as well as the control conditions
stats_filtered = stats[(stats['Cp'] < stats['Cp_no-cDNA']) & ((stats['Cp'] < stats['Cp_no-RT'])) & ~(stats.construct.isin(['no-cDNA','no-RT']))].copy()
display(stats_filtered)

Calculate GAPDH expression for each condition (biorep + construct + dox).

Then one more filtering step: Exclude conditions with low GAPDH expression. This indicates poor RNA extraction (or otherwise low cDNA template).

In [ ]:
index_on = ['exp','biorep','construct','dox','primers']
gapdh = stats_filtered.set_index(index_on)[['Cp']].xs('GAPDH', level='primers').rename(columns={'Cp': 'Cp_GAPDH'})
stats_filtered = stats_filtered.merge(gapdh, on=index_on[:-1], how='left')
stats_filtered.fillna(35, inplace=True)

gapdh_thresh = 21   # set by eye / what is expected for a highly expressed gene
stats_filtered = stats_filtered[stats_filtered['Cp_GAPDH'] < gapdh_thresh]
display(stats_filtered)

Finally we have values for each primer set for each condition (biorep + construct + dox)! Let's plot to double-check.

In [ ]:
plot_df = stats_filtered
g = sns.catplot(data=plot_df, x='primers', y='Cp', col='construct', hue='dox',
                dodge=True, margin_titles=True)
for ax in g.axes.flatten():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

## Calculate normalized expression

First, calculate $\Delta C_p$ relative to the GAPDH condition. This normalizes the RNA extraction / RT reaction and differences in global expression across conditions.

The normalized expression value is $2^{- \Delta C_p}$

In [ ]:
stats_filtered['delta_Cp'] = stats_filtered['Cp'] - stats_filtered['Cp_GAPDH']
stats_filtered['expression_norm'] = 2**(-stats_filtered['delta_Cp'])

What we really care about is how the different conditions compare to each other. This is $\Delta \Delta C_p$, or relative expression.

For ComMAND, we'll compare normalized expression for the circuit relative to the base gene (for each biorep + dox). We tested several circuits here, so we'll add metadata to annotate circuit type based on construct name.

In [ ]:
metadata = pd.read_csv(rd.rootdir/'inputs'/'plasmid-metadata.csv')
display(metadata)

stats_filtered = stats_filtered.merge(metadata, how='left', on='construct')

Conveniently, we used different primer sets for each circuit. So for each primer set, there is only one base gene in the constructs tested.

However, there are a few cases where the base gene is not included (e.g., we excluded that condition earlier because of high Cp values). In this case, let's exclude circuits missing measurements for the corresponding base gene.

In [ ]:
index_on = ['exp','biorep','dox','primers','group']
base_delta_cp = stats_filtered.set_index(index_on)[['delta_Cp']].xs('base', level='group').rename(columns={'delta_Cp': 'delta_Cp_base'})
stats_filtered = stats_filtered.merge(base_delta_cp, on=index_on[:-1], how='left')

stats_circuits = stats_filtered.dropna(subset='delta_Cp_base')
display(stats_circuits)

Finally, we can compute $\Delta \Delta C_p$ and relative expression, $2^{-\Delta \Delta C_p}$

In [ ]:
stats_circuits['delta_delta_Cp'] = stats_circuits['delta_Cp'] - stats_circuits['delta_Cp_base']
stats_circuits['relative_expression'] = 2**(-stats_circuits['delta_delta_Cp'])

In [ ]:
plot_df = stats_circuits
g = sns.catplot(data=plot_df, x='construct', y='delta_delta_Cp', col='primers', hue='dox',
                dodge=True, margin_titles=True)
for ax in g.axes.flatten():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [ ]:
stats_circuits['circuit'] = stats_circuits['ts_kind'].replace({'T': 'CL', 'NT': 'OL', 'na': 'base', '–': '–'})

In [ ]:
plot_df = stats_circuits.sort_values('circuit', key=lambda x: x.map({'–': 0, 'base': 1, 'OL': 2, 'CL': 3}))
g = sns.catplot(data=plot_df, x='circuit', y='relative_expression', col='primers', hue='dox',
                dodge=True, margin_titles=True)

While quick to make, the plots above are a bit hard to read/interpret. Let's make a couple nicer ones to see the final results of our analysis. Bonus points for statistical tests!

In [ ]:
circuit_palette = {'base': 'black', 'OL': 'gray', 'CL': 'teal', '–': 'black'}
star_convert = list(reversed([(1.0, 'n.s.'), (0.05, '*'), (0.01, '**'), (0.001, '***'), (0.0001, '****')]))
pval_to_stars = lambda x: next(val[1] for val in star_convert if val[0] > x)

In [ ]:
f, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw={'wspace': 0.3}, sharex=True, sharey=True)
primer_sets = ['mRuby2-1', 'mRuby2-2', 'FXN']
for ax, primer in zip(axes, primer_sets):
    plot_df = stats_circuits[stats_circuits.primers==primer]
    y = 'relative_expression'
    sns.scatterplot(data=plot_df, x='circuit', y=y, 
                    hue='circuit', palette=circuit_palette, ax=ax, legend=False)
    ax.set(xlim=(-0.5,3.5), title=primer, ylabel='Relative expression, $2^{-\Delta \Delta C_p}$', ylim=(-0.05,1.3))
    xval_order = [l.get_text() for l in ax.get_xticklabels()]

    # T-tests comparing OL to CL circuits
    pair = ('OL','CL')
    x_start = xval_order.index(pair[0])
    x_end = xval_order.index(pair[1])
    y_loc = 0.9
    pad = 0.02
    star = pval_to_stars(sp.stats.ttest_ind(plot_df[(plot_df.circuit==pair[0])][y], plot_df[(plot_df.circuit==pair[1])][y],).pvalue)
    print(f'{pair}: {star}')
    _ = ax.text(x_start+(x_end-x_start)/2, y_loc - pad * (star.startswith('*')), star, 
                transform=ax.get_xaxis_transform(), ha='center', va='center', backgroundcolor='white', bbox={"facecolor": 'white', "pad": 1})
    ax.add_artist(matplotlib.lines.Line2D([x_start, x_end], [y_loc, y_loc], transform=ax.get_xaxis_transform(), color='k', linewidth=1))
    
    fc = plot_df[(plot_df.circuit==pair[1])][y].mean() / plot_df[(plot_df.circuit==pair[0])][y].mean()
    print(f'{fc:0.2f}')
    _ = ax.text(x_start+(x_end-x_start)/2, y_loc+0.03, f'{fc:0.2f}x', size='small',
                transform=ax.get_xaxis_transform(), ha='center', va='bottom', 
                backgroundcolor='white', bbox={"facecolor": 'white', "pad": 1})

In [ ]:
f, axes = plt.subplots(1,4, figsize=(13,5), gridspec_kw={'wspace': 0.3}, sharex=True, sharey=True)
primer_sets = ['EGFP-1', 'EGFP-2', 'FMRP-1', 'FMRP-2']
for ax, primer in zip(axes, primer_sets):
    plot_df = stats_circuits[stats_circuits.primers==primer]
    y = 'relative_expression'
    sns.scatterplot(data=plot_df, x='circuit', y=y, 
                    hue='circuit', palette=circuit_palette, ax=ax, legend=False)
    ax.set(xlim=(-0.5,3.5), title=primer, ylabel='Relative expression, $2^{-\Delta \Delta C_p}$', ylim=(-0.05,3))
    xval_order = [l.get_text() for l in ax.get_xticklabels()]

    # T-tests comparing OL to CL circuits
    pair = ('OL','CL')
    x_start = xval_order.index(pair[0])
    x_end = xval_order.index(pair[1])
    y_loc = 0.9
    pad = 0.02
    star = pval_to_stars(sp.stats.ttest_ind(plot_df[(plot_df.circuit==pair[0])][y], plot_df[(plot_df.circuit==pair[1])][y],).pvalue)
    print(f'{pair}: {star}')
    _ = ax.text(x_start+(x_end-x_start)/2, y_loc - pad * (star.startswith('*')), star, 
                transform=ax.get_xaxis_transform(), ha='center', va='center', backgroundcolor='white', bbox={"facecolor": 'white', "pad": 1})
    ax.add_artist(matplotlib.lines.Line2D([x_start, x_end], [y_loc, y_loc], transform=ax.get_xaxis_transform(), color='k', linewidth=1))
    
    fc = plot_df[(plot_df.circuit==pair[1])][y].mean() / plot_df[(plot_df.circuit==pair[0])][y].mean()
    print(f'{fc:0.2f}')
    _ = ax.text(x_start+(x_end-x_start)/2, y_loc+0.03, f'{fc:0.2f}x', size='small',
                transform=ax.get_xaxis_transform(), ha='center', va='bottom', 
                backgroundcolor='white', bbox={"facecolor": 'white', "pad": 1})

# Example 2: Absolute quantification of a genomics library

For library quantification, we'll implement formulas from [NEB qPCR Library Quantification calculator](https://nebiocalculator.neb.com/#!/qPCRlibQnt)

## Load data

In [ ]:
base_path = rd.datadir/'data'/'qPCR'/'chris'/'2025.09.06-rcmc-lib-quant'
output_path = rd.rootdir/'output'/'qPCR-example'
cache_path = output_path/'data2.gzip'

rd.plot.plot_well_metadata(base_path/'wells.yaml')

In [ ]:
data = pd.DataFrame()
if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = rd.qpcr.load_single_csv_with_metadata(base_path/'cp_table.txt', base_path/'wells.yaml', is_default=True)
    data.dropna(subset='sample', inplace=True)
    data['dilution'] = data['dilution'].astype(float)
    data.to_parquet(rd.outfile(cache_path))
display(data)

Plot samples (not standard curve) to look for outlier points.

In [ ]:
plot_df = data[~(data['sample'].str.contains('standard')) & ~(data['sample']=='NTC')]
f, ax = plt.subplots(1,1, figsize=(len(plot_df['sample'].unique())*0.4, 4))
sns.stripplot(data=plot_df, x='sample', y='Cp', hue='dilution', ax=ax, dodge=True)
sns.move_legend(ax, 'upper left', bbox_to_anchor=(1,1))
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

## Compute standard curve

In [ ]:
# Map standards to known concentrations (pM)
standards_conc = {
    'standard-1': 100,
    'standard-2': 10,
    'standard-3': 1,
    'standard-4': 0.1,
    'standard-5': 0.01,
    'standard-6': 0.001,
    'NTC': 0,
}
standards = data.loc[data.dilution==0, ['sample','Cp']]
standards['conc'] = standards['sample'].map(standards_conc)
standards['log_conc'] = standards.conc.apply(np.log10)

# Perform linear regression on standards
standard_curve = sp.stats.linregress(standards.loc[standards.conc!=0, 'log_conc'], standards.loc[standards.conc!=0, 'Cp'])
print(f'y = {standard_curve.slope:.2f} x + {standard_curve.intercept:.2f}')
print(f'R^2 = {abs(standard_curve.rvalue):.3f}')
efficiency = (10**(-1/standard_curve.slope) - 1)*100 # percentage
print(f'Efficiency: {efficiency:0.1f}% -- ' + ('good!' if (efficiency >= 90) and (efficiency <= 110) else 'outside optimal range'))

# Plot standard curve
f, ax = plt.subplots(1,1, figsize=(6,5))
ax.scatter(x=standards.conc, y=standards.Cp, label='standards', ec='white', lw=0.75)
xs = np.logspace(-3,2,1000)
ys = standard_curve.slope * np.log10(xs) + standard_curve.intercept
ax.plot(xs, ys, color='crimson', label='linear\nregression')
ax.set(xlabel='Conc. (pM)', xlim=(-0.0005,200), ylabel='Cp')
ax.set_xscale('symlog', linthresh=0.001)
ax.legend()
pad = 0.02
ax.annotate(f'$R^2$: {abs(standard_curve.rvalue):0.3f}', (0+pad, 0.1), xycoords='axes fraction',
            ha='left', va='bottom', size='medium')
ax.annotate(f'Efficiency: {efficiency:0.1f}%', (0+pad, 0+pad), xycoords='axes fraction',
            ha='left', va='bottom', size='medium')

Now, use the standard curve to quantify concentrations of samples (libraries).

We'll take the mean of the technical replicates, since these looked good above.

In [ ]:
# Compute sample concentrations from standard curve
def calc_concentration(x, standard_curve):
    return 10**((x.mean()-standard_curve.intercept)/standard_curve.slope)

concentrations = data[~(data['sample'].str.contains('standard')) & ~(data['sample']=='NTC')].groupby(['sample','dilution'])['Cp'].apply(calc_concentration, standard_curve).reset_index().rename(columns={'Cp': 'conc'})
display(concentrations)

## Calculate actual library concentration, adjusting for fragment size

For this experiment, we ran the libraries on the Bioanalyzer. This gives us the average fragment size in the library, which we can use to convert concentration to mass. 

In [ ]:
# From Bioanalyzer (manually type numbers (nM) from pdf of traces)
frags = pd.DataFrame([
    ['lib01', 425, 75.481], ['lib02', 426, 27.807], # UT -dox
    ['lib03', 410, 34.314], ['lib04', 391, 48.973], # Con -dox
    ['lib05', 397, 62.729], ['lib06', 433, 7.873],  # UT +dox
    ['lib07', 434, 25.146], ['lib08', 427, 43.507], # Con +dox
], columns=['sample', 'avg_frag_size', 'measured_conc'])

frags['conc'] = frags['measured_conc'] * 10   # samples were diluted 1:10 for Bioanalyzer run
display(frags)

Since the libraries look similar, let's just use the median across the samples as the average fragment size for all.

Where does the 399 come from? I'm not sure XD We should look this up; maybe NEB has more info.

Finally, we can convert the measured concentration to the actual library concentration using the dilution (and convert this to nM).

In [ ]:
median_frag_size = frags.avg_frag_size.median()
print(f'Median avg. fragment size: {median_frag_size:n} bp')

concentrations['avg_frag_size'] = median_frag_size
concentrations['conc_size_adj'] = concentrations.conc * 399 / concentrations.avg_frag_size
concentrations['lib_conc_size_adj'] = concentrations.conc_size_adj * concentrations.dilution / 1000 # convert from pM to nM
concentrations['lib_conc'] = concentrations.conc * concentrations.dilution / 1000 # convert from pM to nM

We can plot this value for each dilution. These should agree if we pipetted well.

In [ ]:
plot_df = concentrations
f, ax = plt.subplots(1,1, figsize=(len(plot_df['sample'].unique())*0.4, 4))
sns.stripplot(data=plot_df, x='sample', y='lib_conc_size_adj', hue='dilution', ax=ax, dodge=True)
sns.move_legend(ax, 'upper left', bbox_to_anchor=(1,1))
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(ylabel='Size-adjusted conc. (nM)')

Average measurements over the two dilutions to get a single library concentration for each sample (library).

In [ ]:
final_library_conc = concentrations.groupby('sample')[['lib_conc','lib_conc_size_adj']].mean().reset_index()

Note that the Bioanalyzer run also calculated a concentration. This is size-adjusted in nM, which is the same as what we calculated for our final library concentration. Let's plot to compare; we expect these to generally agree, but they may not be perfect.

In [ ]:
f = sns.scatterplot(data=final_library_conc, x='sample', y='lib_conc_size_adj', label='qPCR')
sns.scatterplot(data=frags, x='sample', y='conc', ax=f, label='Bioanalyzer')
_ = f.set_xticklabels(f.get_xticklabels(), rotation=90)
f.set(ylabel='Library conc. (nM)')

Looks like the Bioanalyzer concentrations are pretty variable and different from the qPCR concentrations. Some of the samples had incorrect upper marker detection (i.e., size range in which to quantify) in the Bioanalyzer results, which we forgot to fix, so trust the qPCR values more.

## Compute masses for pooling RCMC libraries

For RCMC, we prepare libraries for each sample, then pool the samples for the region-capture step. The pooling should be equal by mass for each library.

We'll use another formula from NEB to convert moles to mass. This requires the measured concentrations (not the size-adjusted one, as the formula does this) and the known volume of the libraries.

In [ ]:
def convert_moles_to_mass(moles, length):
    """
    For a given amount of DNA in moles, use its length
    to calculate its mass in grams.

    Formula from NEB: 
    g = mol x (bp x 615.94 g/mol/bp + 36.04 g/mol)
     - mass of dsDNA (g) = moles dsDNA x (molecular weight of dsDNA (g/mol))
     - molecular weight of dsDNA = (number of base pairs of dsDNA x average molecular weight of a base pair) + 36.04 g/mol
     - average molecular weight of a base pair = 615.94 g/mol, excluding the water molecule removed during polymerization 
       and assuming deprotonated phosphate hydroxyls
     - the additional 36.04 g/mol accounts for the 2 -OH and 2 -H added back to the ends
     - bases are assumed to be unmodified
    """
    return moles * (length * 615.96 + 36.04)

lib_volume = 25 / 1e6   # convert uL to L
final_library_conc['moles'] = lib_volume * final_library_conc['lib_conc'] / 1e9 # convert nM to moles
final_library_conc['mass_ug'] = final_library_conc['moles'].apply(convert_moles_to_mass, [median_frag_size]) * 1e6
display(final_library_conc)

Lastly, compute the volume to pipet to obtain 500 ng (0.5 µg) per library. This is how much we want in the pool.

In [ ]:
final_library_conc['500ng_vol_uL'] = 0.5 / final_library_conc['mass_ug'] * lib_volume * 1e6
print(f'Pooled library volume: {final_library_conc['500ng_vol_uL'].sum():0.1f} µL')

# Print summary for pipetting
display(final_library_conc[['sample','500ng_vol_uL']])